In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet('baseline_with_lags.parquet')

In [2]:
df['foto_mes'] = pd.to_datetime(df['foto_mes'], format='%Y%m')

In [3]:
df_val = df[df['foto_mes']=='2021-07-01']
df_train = df[df['foto_mes']<'2021-07-01']

In [4]:
df_val.shape, df_train.shape

((164682, 155), (979558, 155))

In [5]:
# X_train = df_train.drop(columns=['target'])
# X_val = df_val.drop(columns=['target'])
# y_train = df_train['target']
# y_val = df_val['target']

In [11]:
X_val = df_val.drop(columns=['target'])
y_val = df_val['target']

In [6]:
%%time
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(df_train)
val_data = TabularDataset(df_val)

CPU times: user 1.64 s, sys: 4.64 s, total: 6.28 s
Wall time: 744 ms


/home/ynesterov/autogluon/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from math import inf

def normalized_ganancia(y_true: np.array,
                      y_pred: np.array) -> float:
    ganancia_df = pd.DataFrame({'prob': y_pred, 'truth': y_true})
    ganancia_df = ganancia_df.sort_values(by='prob', ascending=False).reset_index(drop=True)
    ganancia_df['ganancia_ind'] = np.where(ganancia_df['truth']==1, 273, -7)
    ganancia_df['ganancia_roll'] = ganancia_df['ganancia_ind'].cumsum()
    gan_max = ganancia_df['ganancia_roll'].max()
    ind = ganancia_df[ganancia_df['ganancia_roll']==gan_max].index[0]
    ideal_value = np.sum(np.where(y_val=='B2', 1, 0))*273
    res = np.mean(ganancia_df['ganancia_roll'].iloc[ind-1000:ind+1001])
    normalized_res = (ideal_value-res)/ideal_value
    return normalized_res

ag_mean_squared_error_custom_scorer = make_scorer(name='normalized_ganancia',
                                                  score_func=normalized_ganancia,
                                                  optimum=1,
                                                  greater_is_better=True)

In [ ]:
predictor = TabularPredictor(label='target', eval_metric='f1').fit(
    train_data,
    time_limit=6*3600,
    num_cpus=24,
    num_stack_levels=0,
    fit_weighted_ensemble=False,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231119_234858/"
Beginning AutoGluon training ... Time limit = 21600s
AutoGluon will save models to "AutogluonModels/ag-20231119_234858/"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #85-Ubuntu SMP Fri Jul 7 15:25:09 UTC 2023
Disk Space Avail:   312.23 GB / 982.70 GB (31.8%)
Train Data Rows:    979558
Train Data Columns: 154
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  ['C', 'B2']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = C, class 0 = B2
	Note: For your binary classification, AutoGluon arbitrarily selected which label-

In [14]:
preds = predictor.predict_proba(X_val)
preds = preds['B2']
y_val = np.where(y_val=='B2', 1, 0)

In [15]:
def ganancia_integral(probs: np.array, 
                      y_true: np.array) -> float:
    ganancia_df = pd.DataFrame({'prob': probs, 'truth': y_true})
    ganancia_df = ganancia_df.sort_values(by='prob', ascending=False).reset_index(drop=True)
    ganancia_df['ganancia_ind'] = np.where(ganancia_df['truth']==1, 273, -7)
    ganancia_df['ganancia_roll'] = ganancia_df['ganancia_ind'].cumsum()
    gan_max = ganancia_df['ganancia_roll'].max()
    ind = ganancia_df[ganancia_df['ganancia_roll']==gan_max].index[0]
    print(ind)
    res = np.mean(ganancia_df['ganancia_roll'].iloc[ind-1000:ind+1001])
    return res

In [16]:
# With B2+B1 target engineering
ganancia_integral(preds, y_val)

20318


381046.700149925

In [19]:
# True value
ganancia_integral(preds[1], y_val)

135375.9935535241
